In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn import metrics
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
import keras
from keras.models import load_model
from keras import backend as K
from keras import models
from keras.layers import Dense
import tensorflow as tf
from tensorflow.keras.utils import to_categorical

from keras_tuner.tuners import RandomSearch
import keras_tuner

2022-11-08 22:24:55.115484: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-11-08 22:24:55.115562: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-11-08 22:24:55.241779: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-11-08 22:24:57.270662: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2022-11-08 22:24:57.270926: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: ca

In [2]:
from dataset import load_dataset, load_ica_dataset, load_labels, convert_to_epochs
from features import time_series_features, nonlinear_features, entropy_features, hjorth_features, freq_band_features

In [3]:
num_classes = 2
channels = 32
sfreq = 128

In [4]:
#Filtered from SAM40
dataset_ = load_dataset(raw = False)
dataset = convert_to_epochs(dataset_, channels, sfreq)

#ICA filtered
dataset_ica_ = load_ica_dataset()
dataset_ica = convert_to_epochs(dataset_ica_, channels, sfreq)

In [5]:
features = time_series_features(dataset, channels)
features_ica = time_series_features(dataset_ica, channels)
# freq_bands = np.array([1, 4, 8, 13, 31, 50])
# features = freq_band_features(dataset, channels, sfreq, freq_bands)
# features = nonlinear_features(dataset, channels)
# features = hjorth_features(dataset, channels, sfreq)
# features = entropy_features(dataset, channels, sfreq)
data = features
data_ica = features_ica

(120, 25, 32, 128)
(120, 25, 32, 128)


In [6]:
labels = load_labels()
label = pd.concat([labels['t1_math'], labels['t2_math'],
                  labels['t3_math']]).to_numpy()
label = label.repeat(dataset.shape[1])

# Linear Regression

In [7]:
def LinearRegression(data, label):
    K.clear_session()
    x_train, x_test, y_train, y_test = train_test_split(
    data, label, test_size=0.33, random_state=42)
    scaler = StandardScaler()
    scaler.fit(x_train)
    x_train = scaler.transform(x_train)
    x_test = scaler.transform(x_test)
    lr_clf = LogisticRegression(max_iter=1000).fit(x_train, y_train)
    y_pred = lr_clf.predict(x_test)
    y_true = y_test

    scores_lr = lr_clf.score(x_test, y_test)
    precision_lr = metrics.precision_score(y_true, y_pred, average='macro')
    recall_lr = metrics.recall_score(y_true, y_pred, average='micro')
    f1_score_lr = metrics.f1_score(y_true, y_pred, average='weighted')
    print('accuracy is:', scores_lr)
    print('precision is:', precision_lr)
    print('recall is:', recall_lr)
    print('f1_score is:', f1_score_lr)
    metrics.confusion_matrix(y_true, y_pred)

# KNN Classifier

In [8]:
def KNN(data, label):
    K.clear_session()
    x_train, x_test, y_train, y_test = train_test_split(data, label, test_size=0.33, random_state=42)
    scaler = StandardScaler()
    scaler.fit(x_train)
    x_train = scaler.transform(x_train)
    x_test = scaler.transform(x_test)
    neigh_clf = KNeighborsClassifier(n_neighbors=2)
    neigh_clf.fit(x_train, y_train)
    y_pred = neigh_clf.predict(x_test)
    y_true = y_test

    scores_neigh = neigh_clf.score(x_test, y_test)
    precision_neigh = metrics.precision_score(y_true, y_pred, average='macro')
    recall_neigh = metrics.recall_score(y_true, y_pred, average='micro')
    f1_score_neigh = metrics.f1_score(y_true, y_pred, average='weighted')
    print('accuracy is:', scores_neigh)
    print('precision is:', precision_neigh)
    print('recall is:', recall_neigh)
    print('f1_score is:', f1_score_neigh)
    metrics.confusion_matrix(y_true, y_pred)

# SVM Classifier

In [9]:
def SVM(data, label):
    x_train, x_test, y_train, y_test = train_test_split(
    data, label, test_size=0.33, random_state=42)
    svm_clf = SVC(C=10, kernel='rbf')
    svm_clf.fit(x_train, y_train)
    y_pred = svm_clf.predict(x_test)
    y_pred_train = svm_clf.predict(x_train)
    y_true = y_test

    scores_svm_train = svm_clf.score(x_train, y_train)
    precision_svm_train = metrics.precision_score(y_train, y_pred_train, average='macro')
    recall_svm_train = metrics.recall_score(y_train, y_pred_train, average='micro')
    f1_score_svm_train = metrics.f1_score(y_train, y_pred_train, average='weighted')
    print('accuracy is:', scores_svm_train)
    print('precision is:', precision_svm_train)
    print('recall is:', precision_svm_train)
    print('f1_score is:', f1_score_svm_train)
    metrics.confusion_matrix(y_true, y_pred)


In [10]:
print('Filtered data')
LinearRegression(data,label)
print('\nICA filtered data')
LinearRegression(data_ica,label)

Filtered data
accuracy is: 0.6505050505050505
precision is: 0.6490383236134278
recall is: 0.6505050505050505
f1_score is: 0.6505050505050505

ICA filtered data
accuracy is: 0.6474747474747474
precision is: 0.6458736907771612
recall is: 0.6474747474747474
f1_score is: 0.647350051125969


In [11]:
print('Filtered data')
KNN(data,label)
print('\nICA filtered data')
KNN(data_ica,label)

Filtered data
accuracy is: 0.6161616161616161
precision is: 0.634022736611423
recall is: 0.6161616161616161
f1_score is: 0.5899596648717212

ICA filtered data
accuracy is: 0.5707070707070707
precision is: 0.5724594594594594
recall is: 0.5707070707070707
f1_score is: 0.5426690751092353


In [12]:
print('Filtered data')
SVM(data,label)
print('\nICA filtered data')
SVM(data_ica,label)

Filtered data
accuracy is: 0.8009950248756219
precision is: 0.8017411274654813
recall is: 0.8017411274654813
f1_score is: 0.8006208232559378

ICA filtered data
accuracy is: 0.7442786069651741
precision is: 0.7483465987173614
recall is: 0.7483465987173614
f1_score is: 0.7423439799870053


# Neural Network

In [13]:
K.clear_session()
y_v = label
y_v = to_categorical(y_v, num_classes)
x_train, x_test, y_train, y_test = train_test_split(data_ica, y_v, test_size=0.2, random_state=1)
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.25, random_state=1)

In [14]:
def model_builder(hp):
    model = keras.models.Sequential()
    model.add(keras.Input(shape=(x_train.shape[1],)))

    for i in range(hp.Int('layers', 2, 6)):
      model.add(keras.layers.Dense(units=hp.Int('units_' + str(i), 32, 1024, step=32),
                                    activation=hp.Choice('act_' + str(i), ['relu', 'sigmoid'])))

    model.add(keras.layers.Dense(num_classes, activation='softmax', name='out'))
    
    hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])

    model.compile(optimizer=keras.optimizers.adam_v2.Adam(learning_rate=hp_learning_rate),
                loss = "binary_crossentropy",
                metrics=['accuracy'])
    return model

In [15]:
tuner = RandomSearch(
    model_builder,
    objective = 'val_accuracy',
    max_trials = 5,
    executions_per_trial = 2,
    overwrite=True
)

2022-11-08 22:27:31.053636: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-11-08 22:27:31.054175: W tensorflow/stream_executor/cuda/cuda_driver.cc:263] failed call to cuInit: UNKNOWN ERROR (303)
2022-11-08 22:27:31.055121: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (herman-MacBookAir): /proc/driver/nvidia/version does not exist


In [16]:
tuner.search_space_summary()

Search space summary
Default search space size: 6
layers (Int)
{'default': None, 'conditions': [], 'min_value': 2, 'max_value': 6, 'step': 1, 'sampling': None}
units_0 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 1024, 'step': 32, 'sampling': None}
act_0 (Choice)
{'default': 'relu', 'conditions': [], 'values': ['relu', 'sigmoid'], 'ordered': False}
units_1 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 1024, 'step': 32, 'sampling': None}
act_1 (Choice)
{'default': 'relu', 'conditions': [], 'values': ['relu', 'sigmoid'], 'ordered': False}
learning_rate (Choice)
{'default': 0.01, 'conditions': [], 'values': [0.01, 0.001, 0.0001], 'ordered': True}


In [17]:
tuner.search(x_train, y_train, epochs = 50, validation_data= [x_val, y_val])

Trial 5 Complete [00h 02m 23s]
val_accuracy: 0.8183333277702332

Best val_accuracy So Far: 0.8183333277702332
Total elapsed time: 00h 09m 07s
INFO:tensorflow:Oracle triggered exit


In [18]:
model = tuner.get_best_models(num_models=1)[0]

In [19]:
y_pred = model.predict(x_test)
y_true = y_test
y_pred = np.argmax(y_pred, axis=1)
y_true = np.argmax(y_true, axis=1)

19/19 [==============================] - 0s 8ms/step


In [20]:
scores_dnn = model.evaluate(x_test, y_test, verbose=0)
precision_dnn = metrics.precision_score(y_true, y_pred, average='macro')
recall_dnn = metrics.recall_score(y_true, y_pred, average='micro')
f1_score_dnn = metrics.f1_score(y_true, y_pred, average='weighted')
print('accuracy is:', scores_dnn[1])
print('precision is:', precision_dnn)
print('recall is:', recall_dnn)
print('f1_score is:', f1_score_dnn)
metrics.confusion_matrix(y_true, y_pred)

accuracy is: 0.8333333134651184
precision is: 0.8337315921089192
recall is: 0.8333333333333334
f1_score is: 0.8333777822226667


array([[253,  58],
       [ 42, 247]])